In [ ]:
# 이 코랩파일은 유튜브 https://www.youtube.com/watch?v=lSBX-nMQ8cE 에 올라온 Gradio를 이용한 Llama2 프롬프팅 가이드 ipynb 파일을 일부 수정한 파일(코드)입니다.

# 이 파일에서 수정된 부분은 다음과 같습니다.
# 1. road llm model 변경: Llama2 -> Fine-tuning Gemma-2B   (https://huggingface.co/taeseo06/Gemma-2b-PersonalizedFinetuning)
# 2. 모델을 불러오는 과정에서 GoogleDrive 를 연결하여 드라이브에 저장된 파인튜닝 모델, 토크나이저 직접 로드하는 방식 차용
# 3. 한국어 답변 및 일부 출력 오류를 해결하기 위해 generate(모델 생성)부분 일부 수정

In [1]:
!pip install transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.1 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl 

In [ ]:
# import locale
# locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [4]:
!huggingface-cli whoami

taeseo06


 ---
## 모델, 토크나이저 로드
GoogleDrive를 연결하여 직접 드라이브에 접근해서 파인튜닝 된 모델과 토크나이저를 불러오는 방법과, HuggingFace Pipeline으로 모델을 로드하는 방법으로 나누었습니다.

! 외부에서 시연 및 접근 할 때에는 GoogleDrive 연결이 아닌 HuggingFace 로드가 적합.


### 1️⃣. GoogleDrive에서 직접적으로 모델, 토크나이저 로드

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#@title TrainedModel 폴더에 저장된 파인튜닝 모델이름 -> 시도 횟수
trial = '3' #@param {type: "string"}


In [7]:
# 현재 학습을 시도하는 횟수를 지정해서 각각의 경로마다 변경할 수 있게
Route = '/content/drive/MyDrive/Colab Notebooks/LLM 파인튜닝 프로젝트 (Gemma-2B) 학교 안내 챗봇/SchoolHelperChatbot-Gemma2B-Finetuning/TrainedModel/' + str(trial) + 'st' # 학습된 모델의 기본 루트

%cd $Route

/content/drive/MyDrive/Colab Notebooks/LLM 파인튜닝 프로젝트 (Gemma-2B) 학교 안내 챗봇/SchoolHelperChatbot-Gemma2B-Finetuning/TrainedModel/3st


In [8]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model = AutoModelForCausalLM.from_pretrained("./merged_LLM_Model")
# tokenizer = AutoTokenizer.from_pretrained("./merged_LLM_Model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
from transformers import AutoTokenizer, GemmaForCausalLM

model = GemmaForCausalLM.from_pretrained("google/gemma-7b")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [12]:
prompt = "What is your favorite condiment?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=512)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

KeyboardInterrupt: 

In [13]:
prompt = "What is your favorite condiment?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'What is your favorite condiment?\n\nI love ketchup. I love it on my fries, on my hot dogs, on my hamburgers, on'

In [9]:
# 모델의 생성함수 - AutoModelForCausalLM

def generate_text(prompt, max_length=512):
  # 토크나이징
  input_ids = tokenizer.encode(prompt, return_tensors="pt")

  # 모델 추론
  outputs = model.generate(input_ids=input_ids, max_length=max_length)

  # 토큰을 텍스트로 변환
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return generated_text

# 예시
# prompt = "문태서라는 인물에 관해서 설명해줘"
# generated_text = generate_text(prompt)
# print(f"Generated text: {generated_text}")


In [ ]:
prompt = "안녕"
generated_text = generate_text(prompt)
print(f"Generated text: {generated_text}")

Generated text: 안녕하세요!

I'm looking for a fun and engaging way to learn Korean, and I've heard that watching Korean dramas is a great way to do that. But I'm not sure where to start.

What are some good places to watch Korean dramas?

I'd also like to know if there are any specific recommendations for beginner viewers.

Thank you for your help!


In [ ]:
prompt = "선린인터넷고등학교의 위치를 알려줘"
generated_text = generate_text(prompt)
print(f"Generated text: {generated_text}")

Generated text: 선린인터넷고등학교의 위치를 알려줘요.

선린인터넷고등학교는 경기도 용산구에 위치해있습니다.


In [ ]:
prompt = "남영역에서 선린인터넷고등학교 까지 거리가 어떻게 돼?"
generated_text = generate_text(prompt)
print(f"Generated text: {generated_text}")

Generated text: 남영역에서 선린인터넷고등학교 까지 거리가 어떻게 돼?

남영역에서 선린인터넷고등학교 까지의 거리는 약 10분이 걸리는 거리입니다.


In [ ]:
prompt = "선린인터넷고등학교 주변에는 뭐가있어?"
generated_text = generate_text(prompt)
print(f"Generated text: {generated_text}")

Generated text: 선린인터넷고등학교 주변에는 뭐가있어?

선린인터넷고등학교는 서울시 용산구에 위치해있습니다. 

선린인터넷고등학교 주변에는 다음과 같은 몇 가지가 있습니다.

* 선린중학교
* 선린초등학교
* 선린중학교 교복관
* 선린중학교 급장
* 선린중학교 옥외놀이장
* 선린중학교 체육관
* 선린중학교 학교급실
* 선린중학교 교실
* 선린중학교 밖길


### 2️⃣. HuggingFace에 Pipeline 으로 모델, 토크나이저 로드

In [ ]:
from transformers import AutoTokenizer, pipeline
import transformers
import torch

model = "taeseo06/Gemma-2b-PersonalizedFinetuning" # https://huggingface.co/taeseo06/Gemma-2b-PersonalizedFinetuning


gemma_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
# 모델의 생성함수 - pipeline

def get_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = gemma_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512,
    )
    print("Chatbot:", sequences[0]['generated_text'])

In [ ]:
get_response("안녕")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Chatbot: 안녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕녕


In [ ]:
get_response("선린인터넷고등학교의 위치를 알려줘")

Chatbot: 선린인터넷고등학교의 위치를 알려줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘줘


### Drawbacks of `get_response()`

1. **Lack of Conversation History**: The basic approach does not account for past interactions, making it less effective for maintaining a coherent conversation.
2. **Limited Customization**: The function doesn't allow for advanced prompt formatting or handling system-level instructions.
3. **Not Ready for UI Integration**: This basic approach isn't designed for easy integration with user interface libraries like Gradio.

## Improved Prompts

The right structure of Llama 2 prompts:

```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
```

In [ ]:
# Gemma 모델의 프롬프트 형식으로 추정?


'''
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
'''

### Building the Prompt

Explaining the parameters:
- `message` is the current message we're sending
- `history` is the history of conversation as a list of tupples `[(user_msg1, bot_msg1), (usr_msg2, bot_msg2), ...]`


In [22]:
print(f'{\n}')
f'{\n}'

SyntaxError: f-string expression part cannot include a backslash (<ipython-input-22-ace31f25e3ec>, line 1)

In [ ]:
SYSTEM_PROMPT = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.
<</SYS>>

"""

# Formatting function for message and history
def format_message(message: str, history: list, memory_limit: int = 3) -> str:
    """
    Formats the message and history for the Llama model.

    Parameters:
        message (str): Current message to send.
        history (list): Past conversation history.
        memory_limit (int): Limit on how many past interactions to consider.

    Returns:
        str: Formatted message string
    """
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return SYSTEM_PROMPT + f"<start_of_turn>user{message}<end_of_turn>"

    formatted_message = SYSTEM_PROMPT + f"{history[0][0]} [/INST] {history[0][1]} <end_of_turn>"






    # Handle conversation history
    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    # Handle the current message
    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message



In [10]:
# tokenizer.apply_chat_templat (참조)- https://huggingface.co/docs/transformers/main/en/chat_templating
from transformers import AutoModelForCausalLM, AutoTokenizer

input_text = "선린인터넷고등학교의 정확한 위치를 알려줘"

# 모델과 토크나이저는 위에서 지정하여 로드함

# checkpoint = "HuggingFaceH4/zephyr-7b-beta"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)  # You may want to use bfloat16 and/or move to GPU here

messages = [
    # {
    #     "role": "system",
    #     "content": "당신은 선린인터넷고등학교 학생들을 위해 제작된 학교안내 챗봇입니다. 질문자의 물음에는 항상 학습된 사실에 기반한 내용으로 답변해주세요",
    # },
    {"role": "user", "content": input_text}
 ]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=False, return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]))

<bos><start_of_turn>user
선린인터넷고등학교의 정확한 위치를 알려줘<end_of_turn>



In [ ]:
outputs = model.generate(tokenized_chat, max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
선린인터넷고등학교의 정확한 위치를 알려줘<end_of_turn>
<start_of_turn>model
선린인터넷고등학교는 서울시 용산구 용산로 167길에 위치해있습니다.<eos>


---

In [ ]:
# 새로운 생성함수 합치는 테스트


from transformers import AutoModelForCausalLM, AutoTokenizer

input_text = "선린인터넷고등학교의 정확한 위치를 알려줘"

messages = [
    # {
    #     "role": "system",
    #     "content": "당신은 선린인터넷고등학교 학생들을 위해 제작된 학교안내 챗봇입니다. 질문자의 물음에는 항상 학습된 사실에 기반한 내용으로 답변해주세요",
    # },
    {"role": "user", "content": input_text}
 ]
outputs = model.generate(tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"), max_new_tokens=512)
# print(tokenizer.decode(outputs[0]))
# print(tokenizer.decode(tokenized_chat[0]))

# outputs = model.generate(tokenized_chat, max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
선린인터넷고등학교의 정확한 위치를 알려줘<end_of_turn>
<start_of_turn>model
선린인터넷고등학교는 서울시 용산구 용산로 167길에 위치해있습니다.<eos>


In [ ]:
prompt = "선린인터넷고등학교의 정확한 위치를 알려줘"
generated_text = generate_text(prompt)
print(f"Generated text: {generated_text}")

Generated text: 선린인터넷고등학교의 정확한 위치를 알려줘요.

선린인터넷고등학교는 경기도 용산구 용산1길에 위치해있습니다.


### Getting Responses

We need the function to generate responses.

In [ ]:
# Generate a response from the Llama model
def get_llama_response(message: str, history: list) -> str:
    """
    Generates a conversational response from the Llama model.

    Parameters:
        message (str): User's input message.
        history (list): Past conversation history.

    Returns:
        str: Generated response from the Llama model.
    """
    query = format_message(message, history)
    response = ""

    sequences = llama_pipeline(
        query,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512, # 1024
    )

    # generated_text = sequences[0]['generated_text']
    generated_text = generate_text(query)
    response = generated_text[len(query):]  # Remove the prompt from the output

    print("Chatbot:", response.strip())
    return response.strip()



# message = "안녕"

# query = format_message(message, [])
# response = ""

# sequences = llama_pipeline(
#         query,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         max_length=1024,
#     )

# # generated_text = sequences[0]['generated_text']

# generated_text = generate_text(query)
# response = generated_text[len(query):]  # Remove the prompt from the output

# print("Chatbot:", response.strip())


In [ ]:
# 위에 Llama를 기반으로 한 함수를 Gemma 형식에 맞게 변형하려고 복사한 코드

def get_gemma_response(message: str, history: list) -> str:
    """
    Generates a conversational response from the Llama model.

    Parameters:
        message (str): User's input message.
        history (list): Past conversation history.

    Returns:
        str: Generated response from the Llama model.
    """
    query = format_message(message, history) # format_message 함수를 날리고 다른 함수를 사용했기에 변경필요

    response = ""

    sequences = gemma_pipeline(
        query,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
    )

    # generated_text = sequences[0]['generated_text']
    generated_text = generate_text(query)
    response = generated_text[len(query):]  # Remove the prompt from the output

    print("Chatbot:", response.strip())
    return response.strip()


In [ ]:
prompt = "What is your favorite condiment?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=512)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [15]:
# prompt = "What is your favorite condiment?"
# inputs = tokenizer(prompt, return_tensors="pt")

# Generate
# generate_ids = model.generate(tokenizer("What is your favorite condiment?", return_tensors).input_ids, max_length=30)


tokenizer.batch_decode(model.generate(tokenizer("What is your favorite condiment?", return_tensors="pt").input_ids, max_length=30), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'What is your favorite condiment?\n\nI love ketchup. I love it on my fries, on my hot dogs, on my hamburgers, on'

In [ ]:
import gradio as gr

# gr.ChatInterface(get_llama_response).launch(debug=True)
gr.ChatInterface(get_gemma_response).launch(debug=True)

gr.ChatInterface().launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://73a7ccdb265490cb08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1786, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1336, in call_function
    prediction = await fn(*processed_input)
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 726, in async_wrapper
    response = await f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py", line 507, in _submit_fn
    response = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_async

### Conclusion

Thanks to the Hugging Face Library, creating a pipeline to chat with llama 2 (or any other open-source LLM) is quite easy.

But if you worked a lot with much larger models such as GPT-4, you need to adjust your expectations.